# Courtesy and Insights

https://blog.roboflow.com/instance-segmentation-roboflow/

https://blog.roboflow.com/how-to-train-yolov8-instance-segmentation/

# Step 1. Generate video of fresh and spoiled lemon

# Step 2. Convert video frames to jpeg images

In [ ]:
import cv2
import os
from google.colab import drive


# Mount Google Drive
drive.mount('/content/drive')

# Path to the video file
video_path = '/content/drive/My Drive/Colab Notebooks/{input}.mp4'

# Directory to save the frames locally in Colab
save_dir = 'xxx'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# Capture the video from the file
cap = cv2.VideoCapture(video_path)

current_frame = 0
while True:
    # Read the next frame from the video
    ret, frame = cap.read()

    if not ret:
        print("Finished reading the video.")
        break

    # Save the frame as a JPEG file
    save_path = os.path.join(save_dir, f"frame_{current_frame:04d}.jpg")
    cv2.imwrite(save_path, frame)
    print(f"Saved: {save_path}")

    current_frame += 1

cap.release()

print("All frames are extracted and saved locally.")


# Step 3. Upload images to Roboflow.

Do manual annotation

Add Labels

Add Bounding boxes.

Draw polygon using Smart Polygon tool and Free form polygon tool

Split Images to train, test, valid datasets

https://universe.roboflow.com/test-4h9f7/fruit-spoil-detection#

# Step 4. Install packages

In [9]:
import os
HOME = os.getcwd()
print(HOME)

/content


In [10]:
!pip install ultralytics==8.0.196 -q

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.196 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 33.6/78.2 GB disk)


In [11]:
from ultralytics import YOLO

from IPython.display import display, Image

In [12]:
!apt-get install ffmpeg

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [13]:
import getpass
import os
YOUR_API_KEY = getpass.getpass()

··········


In [14]:

!mkdir {HOME}/datasets
%cd {HOME}/datasets

!pip install roboflow --quiet


/content/datasets
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 4.7 MB/s eta 0:00:00


In [15]:
import requests

#YOUR_API_KEY = "YOUR_ACTUAL_API_KEY"  # Replace with your actual API key

url = "https://api.roboflow.com/"
headers = {"Authorization": f"Bearer {YOUR_API_KEY}"}

response = requests.get(url, headers=headers)

if response.status_code == 200:
    workspaces = response.json()
    print(workspaces)
else:
    print(f"Error: {response.status_code}")
    print(response.text)

{'welcome': 'Welcome to the Roboflow API.', 'instructions': 'You are successfully authenticated.', 'docs': 'https://docs.roboflow.com', 'workspace': 'test-4h9f7'}


# Step 5. Download Dataset from roboflow

In [16]:

from roboflow import Roboflow
rf = Roboflow(api_key=YOUR_API_KEY)



project = rf.workspace("test-4h9f7").project("fruit-spoil-detection")
version = project.version(1)
dataset = version.download("yolov8")






loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to fruit-spoil-detection-1 in yolov8:: 100%|██████████| 630/630 [00:00<00:00, 1590.63it/s]


In [17]:
!ls -ltr /content/datasets/

total 4
drwxr-xr-x 5 root root 4096 Aug 14 05:31 fruit-spoil-detection-1


# Step 6. Train ultralytics YOLO model

In [18]:
from ultralytics import YOLO

model = YOLO('yolov8m-seg.pt')

results = model.train(data=f"{dataset.location}/data.yaml", epochs=50, imgsz=640)

100%|██████████| 52.4M/52.4M [00:01<00:00, 28.6MB/s]
New https://pypi.org/project/ultralytics/8.2.77 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.196 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=segment, mode=train, model=yolov8m-seg.pt, data=/content/datasets/fruit-spoil-detection-1/data.yaml, epochs=50, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_

In [19]:
#!ls -ltr /content/datasets/

# Step 7. Run Prediction on test images

In [20]:
from ultralytics import YOLO
import os

# Set up paths
home_dir = "/content/datasets"
model_path = os.path.join(home_dir, "runs/segment/train/weights/best.pt")
image_dir = os.path.join("/content/datasets/fruit-spoil-detection-1", "test/images")

# Load the model
model = YOLO(model_path)

# Run prediction
results = model.predict(source=image_dir, conf=0.25, save=True)

# If you need to access the output, you can iterate through results
for result in results:
    print(result)  # or save, or visualize



image 1/13 /content/datasets/fruit-spoil-detection-1/test/images/frame_0001_jpg.rf.633cc298dfd9fe8c050e106f18c84578.jpg: 640x640 2 freshs, 3 spoileds, 49.3ms
image 2/13 /content/datasets/fruit-spoil-detection-1/test/images/frame_0003_jpg.rf.32f2035a05a361a52160107e11708e55.jpg: 640x640 2 freshs, 2 spoileds, 39.1ms
image 3/13 /content/datasets/fruit-spoil-detection-1/test/images/frame_0006_jpg.rf.5fc834bf7cbd52c8a739a5ed69b34e37.jpg: 640x640 2 freshs, 2 spoileds, 38.7ms
image 4/13 /content/datasets/fruit-spoil-detection-1/test/images/frame_0010_jpg.rf.b8a04b4f7c4f6c731cc72b9644de6d3a.jpg: 640x640 2 freshs, 2 spoileds, 36.0ms
image 5/13 /content/datasets/fruit-spoil-detection-1/test/images/frame_0011_jpg.rf.92590df2f8ad2472d3e950efe37c2014.jpg: 640x640 2 freshs, 2 spoileds, 36.2ms
image 6/13 /content/datasets/fruit-spoil-detection-1/test/images/frame_0043_jpg.rf.9fce5495d416466ab7531d630e167ecd.jpg: 640x640 2 freshs, 3 spoileds, 35.6ms
image 7/13 /content/datasets/fruit-spoil-detection-

ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: ultralytics.engine.results.Masks object
names: {0: 'fresh', 1: 'spoiled'}
orig_img: array([[[147, 145, 151],
        [166, 164, 170],
        [150, 148, 154],
        ...,
        [ 89,  92, 100],
        [120, 123, 131],
        [112, 115, 123]],

       [[155, 153, 159],
        [162, 160, 166],
        [161, 159, 165],
        ...,
        [112, 115, 123],
        [153, 156, 164],
        [137, 140, 148]],

       [[150, 148, 154],
        [146, 144, 150],
        [173, 171, 177],
        ...,
        [111, 114, 122],
        [144, 147, 155],
        [152, 155, 163]],

       ...,

       [[151, 147, 142],
        [181, 177, 172],
        [208, 204, 199],
        ...,
        [203, 196, 211],
        [169, 162, 177],
        [165, 158, 173]],

       [[104, 100,  95],
        [139, 135, 130],
        [205, 201, 196],
        ...,
        [207, 200, 215],


In [21]:
#!ls -ltr /content/datasets/

In [ ]:
import glob
from IPython.display import Image, display

for image_path in glob.glob(f'runs/segment/predict/*.jpg'):
      print(image_path)
      display(Image(filename=image_path, height=600))
      print("\n")

![spoil_prediction_1](spoil_prediction_1.png)

![spoil_prediction_2](spoil_prediction_2.png)

![spoil_prediction_3](spoil_prediction_3.png)

![spoil_prediction_4](spoil_prediction_4.png)

![spoil_with_mask_1](spoil_with_mask_1.png)

![spoil_with_mask_2](spoil_with_mask_2.png)

![spoil_with_mask_3](spoil_with_mask_3.png)


# Step 7. Visualize masks

import matplotlib.pyplot as plt
import cv2

# Assuming `results` is a list of `Results` objects from the YOLO model prediction

for result in results:
    # Load the original image
    img = cv2.imread(result.path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Display the original image
    plt.figure(figsize=(10, 10))
    plt.subplot(1, 2, 1)
    plt.imshow(img_rgb)
    plt.title("Original Image with Bounding Boxes")

    # Draw bounding boxes
    for box in result.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        label = result.names[int(box.cls)]
        cv2.rectangle(img_rgb, (x1, y1), (x2, y2), color=(255, 0, 0), thickness=2)
        cv2.putText(img_rgb, label, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

    plt.imshow(img_rgb)

    # Display the mask on the original image
    plt.subplot(1, 2, 2)
    plt.imshow(img_rgb)
    for mask in result.masks.data:
        mask = mask.cpu().numpy()
        plt.imshow(mask, alpha=0.5, cmap='jet')  # Overlay the mask on the image

    plt.title("Image with Masks")
    plt.show()


In [24]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt

# Set up paths
model_path = "/content/datasets/runs/segment/train/weights/best.pt"
image_path = "/content/drive/My Drive/Colab Notebooks/spoiled_fruit_test1.jpg"
output_image_path = "/content/new_test_img_with_results.jpg"

# Load the model
model = YOLO(model_path)

# Run prediction
results = model.predict(source=image_path, conf=0.25, save=True)





image 1/1 /content/drive/My Drive/Colab Notebooks/spoiled_fruit_test1.jpg: 640x480 2 freshs, 172.8ms
Speed: 4.7ms preprocess, 172.8ms inference, 5.9ms postprocess per image at shape (1, 3, 640, 480)
Results saved to runs/segment/predict2


In [ ]:
import glob
from IPython.display import Image, display

for image_path in glob.glob(f'runs/segment/predict2/*.jpg'):
      print(image_path)
      display(Image(filename=image_path, height=600))
      print("\n")

In [ ]:
import matplotlib.pyplot as plt
import cv2

for result in results:
    # Load the original image
    img = cv2.imread(result.path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Lighten the background image
    img_rgb = cv2.convertScaleAbs(img_rgb, alpha=1.5, beta=50)  # Increase brightness

    # Display the original image with bounding boxes
    plt.figure(figsize=(12, 6))

    plt.subplot(1, 2, 1)
    plt.imshow(img_rgb)
    plt.title("Original Image with Bounding Boxes")

    # Draw bounding boxes
    for box in result.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        label = result.names[int(box.cls)]
        confidence = box.conf[0]
        cv2.rectangle(img_rgb, (x1, y1), (x2, y2), color=(255, 0, 0), thickness=2)
        cv2.putText(img_rgb, f'{label} {confidence:.2f}', (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

    plt.imshow(img_rgb)

    # Display the mask on the lightened image
    plt.subplot(1, 2, 2)
    plt.imshow(img_rgb)
    for mask in result.masks.data:
        mask = mask.cpu().numpy()
        plt.imshow(mask, alpha=0.5, cmap='jet')  # Overlay the mask on the image

    plt.title("Image with Masks")
    plt.axis("off")
    plt.show()


![spoil_with_mask_4](spoil_with_mask_4.png)

In [ ]:
import matplotlib.pyplot as plt
import cv2
import numpy as np

for result in results:
    # Load the original image
    img = cv2.imread(result.path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Display the original image with bounding boxes
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.imshow(img_rgb)
    plt.title("Original Image with Bounding Boxes")

    # # Draw bounding boxes
    # for box in result.boxes:
    #     x1, y1, x2, y2 = map(int, box.xyxy[0])
    #     label = result.names[int(box.cls)]
    #     confidence = box.conf[0]
    #     cv2.rectangle(img_rgb, (x1, y1), (x2, y2), color=(255, 0, 0), thickness=2)
    #     cv2.putText(img_rgb, f'{label} {confidence:.2f}', (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

        # Draw bounding boxes
    for box in result.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        label = result.names[int(box.cls)]
        confidence = box.conf[0]
        cv2.rectangle(img_rgb, (x1, y1), (x2, y2), color=(255, 0, 0), thickness=2)
        cv2.putText(img_rgb, f'{label} {confidence:.2f}', (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

    plt.imshow(img_rgb)

    # Prepare to display masks
    plt.subplot(1, 2, 2)
    plt.imshow(img_rgb)  # Display the original image for mask overlay
    plt.title("Image with Masks")

    # Overlay masks
    for mask in result.masks.data:
        mask = mask.cpu().numpy()
        # Resize mask to match the image dimensions
        mask_resized = cv2.resize(mask, (img_rgb.shape[1], img_rgb.shape[0]), interpolation=cv2.INTER_NEAREST)
        # Apply color map
        mask_colored = cv2.applyColorMap((mask_resized * 255).astype(np.uint8), cv2.COLORMAP_SPRING)
        # Blend the mask with the image
        img_with_mask = cv2.addWeighted(img_rgb, 1, mask_colored, 0.6, 0)  # Adjust alpha for visibility

    plt.imshow(img_with_mask)
    plt.axis("off")
    plt.show()


![spoil_with_mask_5](spoil_with_mask_5.png)

In [29]:
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt

# Set up paths
model_path = "/content/datasets/runs/segment/train/weights/best.pt"
image_path = "/content/drive/My Drive/Colab Notebooks/spoiled_fruit_test2.jpg"
output_image_path = "/content/new_test_img_with_results.jpg"

# Load the model
model = YOLO(model_path)

# Run prediction
results = model.predict(source=image_path, conf=0.25, save=True)



image 1/1 /content/drive/My Drive/Colab Notebooks/spoiled_fruit_test2.jpg: 480x640 2 spoileds, 93.5ms
Speed: 3.2ms preprocess, 93.5ms inference, 4.7ms postprocess per image at shape (1, 3, 480, 640)
Results saved to runs/segment/predict3


In [ ]:
import glob
from IPython.display import Image, display

for image_path in glob.glob(f'runs/segment/predict3/*.jpg'):
      print(image_path)
      display(Image(filename=image_path, height=600))
      print("\n")

![spoil_fruit_test_2](spoil_fruit_test_2.png)

In [ ]:
import matplotlib.pyplot as plt
import cv2

for result in results:
    # Load the original image
    img = cv2.imread(result.path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Lighten the background image
    img_rgb = cv2.convertScaleAbs(img_rgb, alpha=1.5, beta=50)  # Increase brightness

    # Display the original image with bounding boxes
    plt.figure(figsize=(12, 6))

    plt.subplot(1, 2, 1)
    plt.imshow(img_rgb)
    plt.title("Original Image with Bounding Boxes")

    # Draw bounding boxes
    for box in result.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        label = result.names[int(box.cls)]
        confidence = box.conf[0]
        cv2.rectangle(img_rgb, (x1, y1), (x2, y2), color=(255, 0, 0), thickness=2)
        cv2.putText(img_rgb, f'{label} {confidence:.2f}', (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

    plt.imshow(img_rgb)

    # Display the mask on the lightened image
    plt.subplot(1, 2, 2)
    plt.imshow(img_rgb)
    for mask in result.masks.data:
        mask = mask.cpu().numpy()
        plt.imshow(mask, alpha=0.5, cmap='jet')  # Overlay the mask on the image

    plt.title("Image with Masks")
    plt.axis("off")
    plt.show()


![spoil_fruit_test_2](spoil_fruit_test_2_mask.png)

In [32]:
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt

# Set up paths
model_path = "/content/datasets/runs/segment/train/weights/best.pt"
image_path = "/content/drive/My Drive/Colab Notebooks/spoiled_fruit_test3.jpg"
output_image_path = "/content/new_test_img_with_results.jpg"

# Load the model
model = YOLO(model_path)

# Run prediction
results = model.predict(source=image_path, conf=0.25, save=True)




image 1/1 /content/drive/My Drive/Colab Notebooks/spoiled_fruit_test3.jpg: 480x640 2 freshs, 1 spoiled, 39.5ms
Speed: 3.1ms preprocess, 39.5ms inference, 4.6ms postprocess per image at shape (1, 3, 480, 640)
Results saved to runs/segment/predict4


In [ ]:
import glob
from IPython.display import Image, display

for image_path in glob.glob(f'runs/segment/predict4/*.jpg'):
      print(image_path)
      display(Image(filename=image_path, height=600))
      print("\n")

![spoiled_fruit_test3](spoiled_fruit_test3.png)

In [ ]:
import matplotlib.pyplot as plt
import cv2

for result in results:
    # Load the original image
    img = cv2.imread(result.path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Lighten the background image
    img_rgb = cv2.convertScaleAbs(img_rgb, alpha=1.5, beta=50)  # Increase brightness

    # Display the original image with bounding boxes
    plt.figure(figsize=(12, 6))

    plt.subplot(1, 2, 1)
    plt.imshow(img_rgb)
    plt.title("Original Image with Bounding Boxes")

    # Draw bounding boxes
    for box in result.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        label = result.names[int(box.cls)]
        confidence = box.conf[0]
        cv2.rectangle(img_rgb, (x1, y1), (x2, y2), color=(255, 0, 0), thickness=2)
        cv2.putText(img_rgb, f'{label} {confidence:.2f}', (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

    plt.imshow(img_rgb)

    # Display the mask on the lightened image
    plt.subplot(1, 2, 2)
    plt.imshow(img_rgb)
    for mask in result.masks.data:
        mask = mask.cpu().numpy()
        plt.imshow(mask, alpha=0.5, cmap='jet')  # Overlay the mask on the image

    plt.title("Image with Masks")
    plt.axis("off")
    plt.show()


![spoiled_fruit_test3](spoiled_fruit_test3_mask.png)

In [ ]:
import matplotlib.pyplot as plt
import cv2
import numpy as np

for result in results:
    # Load the original image
    img = cv2.imread(result.path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Display the original image with bounding boxes
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.imshow(img_rgb)
    plt.title("Original Image with Bounding Boxes")

    # # Draw bounding boxes
    # for box in result.boxes:
    #     x1, y1, x2, y2 = map(int, box.xyxy[0])
    #     label = result.names[int(box.cls)]
    #     confidence = box.conf[0]
    #     cv2.rectangle(img_rgb, (x1, y1), (x2, y2), color=(255, 0, 0), thickness=2)
    #     cv2.putText(img_rgb, f'{label} {confidence:.2f}', (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

        # Draw bounding boxes
    for box in result.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        label = result.names[int(box.cls)]
        confidence = box.conf[0]
        cv2.rectangle(img_rgb, (x1, y1), (x2, y2), color=(255, 0, 0), thickness=2)
        cv2.putText(img_rgb, f'{label} {confidence:.2f}', (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

    plt.imshow(img_rgb)

    # Prepare to display masks
    plt.subplot(1, 2, 2)
    plt.imshow(img_rgb)  # Display the original image for mask overlay
    plt.title("Image with Masks")

    # Overlay masks
    for mask in result.masks.data:
        mask = mask.cpu().numpy()
        # Resize mask to match the image dimensions
        mask_resized = cv2.resize(mask, (img_rgb.shape[1], img_rgb.shape[0]), interpolation=cv2.INTER_NEAREST)
        # Apply color map
        mask_colored = cv2.applyColorMap((mask_resized * 255).astype(np.uint8), cv2.COLORMAP_SPRING)
        # Blend the mask with the image
        img_with_mask = cv2.addWeighted(img_rgb, 1, mask_colored, 0.6, 0)  # Adjust alpha for visibility

    plt.imshow(img_with_mask)
    plt.axis("off")
    plt.show()


![spoiled_fruit_test3](spoiled_fruit_test3_mask_color.png)

Instance Segmentation

Individual Objects: Instance segmentation identifies each individual object in an image and delineates its boundaries. This means that each object of the same category is distinctly segmented with a unique identifier.

Usage in YOLO: In more advanced versions of YOLO (like YOLOv3 onwards and in the YOLOv8 family), capabilities have been extended beyond bounding box predictions to include segmentation masks, which can be used for instance segmentation. This involves not only locating objects within an image but also segmenting each instance of the objects.

Semantic Segmentation

Category Labeling: Semantic segmentation involves dividing an image into regions based on category but does not differentiate between different instances of the same category. Every pixel is labeled with a class, but individual objects are not uniquely identified.

General Approach: This type of segmentation would label all pixels of a particular class with the same color without distinguishing between different objects of the same class in the image.

Given that the masks are applied per detected object in your code and include unique bounding boxes for each detected instance, it suggests the model is performing instance segmentation. This method is more detailed than semantic segmentation as it provides not only the class label but also distinguishes between different objects of the same class.

# Step 8. Run Prediction on test videos

In [37]:
import cv2
from ultralytics import YOLO

# Initialize the video capture and the YOLO model
video_path = "/content/drive/My Drive/Colab Notebooks/spoiled_fruit_input_test_video.mp4"
cap = cv2.VideoCapture(video_path)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Video writer to save output
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('/content/output_video.mp4', fourcc, fps, (frame_width, frame_height))

model = YOLO("/content/datasets/runs/segment/train/weights/best.pt")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Predict using YOLO
    results = model.predict(source=frame, conf=0.25)

    # Process each result
    for result in results:
        if hasattr(result, 'boxes'):
            boxes = result.boxes.cpu().numpy()  # Move boxes to CPU and convert to numpy array
            for box in boxes:
                r = box.xyxy[0].astype(int)  # Rectangle coordinates
                class_id = int(box.cls[0])  # Class ID
                class_name = model.names[class_id]  # Get class name using the class ID

                # Draw bounding box and label on the frame
                cv2.rectangle(frame, (r[0], r[1]), (r[2], r[3]), (0, 255, 0), 2)
                cv2.putText(frame, f"{class_name} {box.conf[0]:.2f}", (r[0], r[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    # Write the processed frame to the output video
    out.write(frame)

# Release resources
cap.release()
out.release()



0: 384x640 1 fresh, 2 spoileds, 163.5ms
Speed: 3.3ms preprocess, 163.5ms inference, 6.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 fresh, 3 spoileds, 37.6ms
Speed: 5.8ms preprocess, 37.6ms inference, 6.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 fresh, 3 spoileds, 35.3ms
Speed: 3.3ms preprocess, 35.3ms inference, 7.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 spoileds, 38.7ms
Speed: 5.3ms preprocess, 38.7ms inference, 5.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 spoileds, 38.0ms
Speed: 10.5ms preprocess, 38.0ms inference, 7.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 spoileds, 36.4ms
Speed: 9.4ms preprocess, 36.4ms inference, 5.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 spoileds, 39.8ms
Speed: 3.1ms preprocess, 39.8ms inference, 6.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 spoileds, 36.0ms
Speed: 3.5ms preprocess, 36.0ms inference, 

In [1]:
!ffmpeg -i output_video.mp4 -vcodec libx264 -y output_video_conv.mp4


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

# Step 10. Display Inference Video

In [ ]:
from IPython.display import HTML
from base64 import b64encode

# Function to display the processed video
def display_video(video_path):
    mp4 = open(video_path,'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    return HTML("""
        <video width=700 controls>
            <source src="%s" type="video/mp4">
        </video>
    """ % data_url)

# Display the processed video
display_video("output_video_conv.mp4")

![spoil_video_out_1](spoil_video_out_1.png)

![spoil_video_out_2](spoil_video_out_2.png)

In [39]:
from google.colab import files
output_path='/content/output_video.mp4'
# Download the processed video file
files.download(output_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Step 11. Display Inference Video with masks

In [43]:
import cv2
import numpy as np

# Initialize the video capture and the YOLO model
video_path = "/content/drive/My Drive/Colab Notebooks/small_spoiled_fruit_input_test_video.mp4"
video_path = "/content/drive/My Drive/Colab Notebooks/spoiled_fruit_input_test_video.mp4"

cap = cv2.VideoCapture(video_path)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Video writer to save output
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('/content/output_video_with_mask.mp4', fourcc, fps, (frame_width, frame_height))

model = YOLO("/content/datasets/runs/segment/train/weights/best.pt")  # Ensure the model path is correct

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Predict using the model
    results = model.predict(source=frame, conf=0.25)  # Adjust confidence as needed

    # Process each result
    for result in results:
        if hasattr(result, 'masks') and result.masks is not None:
            for mask in result.masks.data:
                mask = mask.cpu().numpy()
                # Resize mask to match the frame dimensions
                mask_resized = cv2.resize(mask, (frame_width, frame_height), interpolation=cv2.INTER_NEAREST)
                # Create a color overlay only where the mask is positive
                color_mask = np.zeros_like(frame)
                color_mask[mask_resized > 0.5] = [0, 255, 0]  # Change [0, 255, 0] to any color you prefer

                # Blend the color mask with the original frame
                frame = cv2.addWeighted(frame, 1, color_mask, 0.4, 0)  # Adjust alpha to control transparency

    # Write the processed frame to the output video
    out.write(frame)

# Release resources
cap.release()
out.release()
print("Video processing complete. Output saved.")



0: 384x640 1 fresh, 2 spoileds, 34.8ms
Speed: 2.9ms preprocess, 34.8ms inference, 4.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 fresh, 3 spoileds, 30.4ms
Speed: 3.8ms preprocess, 30.4ms inference, 4.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 fresh, 3 spoileds, 30.2ms
Speed: 3.6ms preprocess, 30.2ms inference, 4.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 spoileds, 30.2ms
Speed: 6.1ms preprocess, 30.2ms inference, 4.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 spoileds, 30.2ms
Speed: 3.6ms preprocess, 30.2ms inference, 4.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 spoileds, 30.2ms
Speed: 3.5ms preprocess, 30.2ms inference, 4.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 spoileds, 30.2ms
Speed: 3.4ms preprocess, 30.2ms inference, 4.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 spoileds, 30.2ms
Speed: 3.4ms preprocess, 30.2ms inference, 4.3

Video processing complete. Output saved.


In [44]:
from google.colab import files
output_path='/content/output_video_with_mask.mp4'
# Download the processed video file
files.download(output_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!ls -ltr /content

In [ ]:
!apt-get install ffmpeg

In [3]:
!ffmpeg -i output_video_with_mask.mp4 -vcodec libx264 -y output_video_with_mask_conv.mp4




ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [ ]:
from IPython.display import HTML
from base64 import b64encode

# Function to display the processed video
def display_video(video_path):
    mp4 = open(video_path,'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    return HTML("""
        <video width=700 controls>
            <source src="%s" type="video/mp4">
        </video>
    """ % data_url)

# Display the processed video
display_video("output_video_with_mask_conv.mp4")

![Sample Output](spoil_video_out_3_mask.png)

![Sample Output](spoil_video_out_4_mask.png)


# You can watch the full video here


https://github.com/AIBotTeachesAI/Segmentation_Spoiled_Fruit_YOLO/blob/main/README.md